In [22]:
import pandas as pd
import numpy as ny

In [23]:
county_name_match=pd.read_excel("/home/jian/SunnyD/Otherinput/CountyMatch.xlsx",sheetname="Matching County")
del county_name_match['x']
county_name_match.columns=['County_St_IRI','County_St_Nielsen']

In [24]:
nielsen_dma=pd.read_excel("/home/jian/SunnyD/Otherinput/Zips by DMA 16-17.xlsx",skiprows=1)
nielsen_dma=nielsen_dma.iloc[:,[2,6,7,0]]
nielsen_dma.columns=['DMA','ST','County','zip_cd']
nielsen_dma['County_St_Nielsen']=nielsen_dma['County'].apply(lambda x: x.replace(" ", ""))
nielsen_dma['County_St_Nielsen']=nielsen_dma['County_St_Nielsen']+", "+nielsen_dma['ST']

In [25]:
IRI_dma=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",skiprows=1,sheetname="U.S. Counties")
IRI_dma=IRI_dma.iloc[:,[1,2,3]]
IRI_dma.columns=['County','ST','IRI_Market']
IRI_dma=IRI_dma[IRI_dma['IRI_Market']!="NO MARKET"]
IRI_dma['County_St_IRI']=IRI_dma['County'].apply(lambda x: x.replace(" ", ""))
IRI_dma['County_St_IRI']=IRI_dma['County_St_IRI']+", "+IRI_dma['ST']
IRIMKT_NielsenCounty=pd.merge(county_name_match,IRI_dma,on="County_St_IRI",how="left")
IRIMKT_NielsenCounty=IRIMKT_NielsenCounty[['IRI_Market','County_St_Nielsen']].drop_duplicates()

In [26]:
# Test result = No duplicate in the IRI markets
'''
All_Overlap=pd.DataFrame(columns=['County_St_Nielsen','DMA','Common_Zips','Zip_Count_In_DMA'])
County_DMA=pd.DataFrame(columns=['County_St_Nielsen','DMA'])

for county in IRIMKT_NielsenCounty['County_St_Nielsen'].unique():
    df=nielsen_dma[nielsen_dma['County_St_Nielsen']==county]

    if len(df['DMA'].unique())==0:
        error=[{'County_St_Nielsen' : county, 'DMA': 'Error'},]
        df_app=pd.DataFrame(error)
        
    elif len(df['DMA'].unique())==1:
        df_app=df[['County_St_Nielsen','DMA']].drop_duplicates()
        
    elif len(df['DMA'].unique())>1:
        zip_county=df['zip_cd'].unique()
        Zip_Overlap=pd.DataFrame(columns=['County_St_Nielsen','DMA','Common_Zips','Zip_Count_In_DMA'])
        
        for i in range(len(df['DMA'].unique())):
            zip_DMA=nielsen_dma[nielsen_dma['DMA']==df['DMA'].unique()[i]]['zip_cd'].unique()
            common_zips=pd.Series(list(set(zip_county).intersection(set(zip_DMA))))
            Zip_Count_In_DMA=len(common_zips)
            DMA_ZIP_OVERLAP=pd.DataFrame[{'County_St_Nielsen':county,'DMA':df['DMA'].unique()[i],'Common_Zips':common_zips,'Zip_Count_In_DMA':Zip_Count_In_DMA}]
            Zip_Overlap=Zip_Overlap.append(DMA_ZIP_OVERLAP)
            
        df_app=Zip_Overlap[Zip_Overlap['Zip_Count_In_DMA']==max(Zip_Overlap['Zip_Count_In_DMA'])]
        df_app=df[['County_St_Nielsen','DMA']]
        
        All_Overlap=All_Overlap.append(Zip_Overlap)
        
    County_DMA=County_DMA.append(df_app)
# No County in more than one DMA
'''        
County_DMA=nielsen_dma[nielsen_dma['County_St_Nielsen'].isin(IRIMKT_NielsenCounty['County_St_Nielsen'])]
County_DMA=County_DMA[['DMA','County_St_Nielsen']].drop_duplicates()
County_DMA.shape            


(946, 2)

In [27]:
IRI_to_dma=pd.merge(IRIMKT_NielsenCounty,County_DMA,on="County_St_Nielsen",how="left")
IRI_with_DMA=IRI_to_dma.groupby(['IRI_Market'])['DMA'].apply(set).to_frame()
IRI_with_DMA['DMA']=IRI_with_DMA['DMA'].apply(list)
IRI_with_DMA.reset_index(inplace=True)
IRI_with_DMA['DMA_count']=IRI_with_DMA['DMA'].apply(lambda x: len(x))

In [28]:
IRI_with_DMA.to_excel("/home/jian/SunnyD/Otherinput/IRI_DMA.xlsx",index=False)

# Add Percentage of zips for each DMA associated with an IRI Market

In [29]:
nielsen_county_zip=nielsen_dma[['County_St_Nielsen','zip_cd']]
nielsen_county_zip=nielsen_county_zip.groupby(['County_St_Nielsen'])['zip_cd'].apply(set).to_frame()
nielsen_county_zip.reset_index(inplace=True)
nielsen_county_zip['zip_cd']=nielsen_county_zip['zip_cd'].apply(list)
IRIMKT_Zip=pd.merge(IRIMKT_NielsenCounty,nielsen_county_zip,on='County_St_Nielsen',how='left')
IRIMKT_Zip_Agg=IRIMKT_Zip.groupby(['IRI_Market'])['zip_cd'].apply(sum).to_frame()
IRIMKT_Zip_Agg.reset_index(inplace=True)
IRIMKT_Zip_Agg['zip_cd']=IRIMKT_Zip_Agg['zip_cd'].apply(set)

In [30]:
DMA_Zip_Agg=nielsen_dma.groupby(['DMA'])['zip_cd'].apply(set).to_frame()
DMA_Zip_Agg.reset_index(inplace=True)

In [31]:
df_app=pd.DataFrame(columns=['IRI_Market','DMA'])
for i in range(len(IRI_with_DMA)):
    IRI_Market=IRI_with_DMA['IRI_Market'][i]
    zips_of_IRI=IRIMKT_Zip_Agg[IRIMKT_Zip_Agg['IRI_Market']==IRI_Market]
    zips_of_IRI.reset_index(inplace=True)
    zips_of_IRI=zips_of_IRI['zip_cd'][0]


    list_dma_int=[]
    for j in range(len(IRI_with_DMA['DMA'][i])):
        DMA=IRI_with_DMA['DMA'][i][j]
        zips_of_DMA=DMA_Zip_Agg[DMA_Zip_Agg['DMA']==DMA]
        zips_of_DMA.reset_index(inplace=True)
        zips_of_DMA=zips_of_DMA['zip_cd'][0]

        overlap_of_zips=pd.Series(list(zips_of_DMA.intersection(zips_of_IRI)))
        pctg_overlap=round(len(overlap_of_zips)/len(zips_of_DMA),4)
        list_intersec=[{DMA:pctg_overlap}]
        list_dma_int=list_dma_int+list_intersec
    df_app=df_app.append(pd.DataFrame([{'IRI_Market':IRI_Market,'DMA':list_dma_int}]))
IRI_Market_DMA=df_app[['IRI_Market','DMA']]
IRI_Market_DMA['DMA_Count']=IRI_Market_DMA['DMA'].apply(len)
IRI_Market_DMA.to_excel("/home/jian/SunnyD/Otherinput/IRI_DMA_Pctg.xlsx",index=False)


        
        

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# All IRI Market with Zips

In [32]:
zip_all_in_IRIMKT=nielsen_dma[nielsen_dma['County_St_Nielsen'].isin(IRIMKT_NielsenCounty['County_St_Nielsen'])]
zip_all_in_IRIMKT=pd.merge(zip_all_in_IRIMKT,IRIMKT_NielsenCounty,on='County_St_Nielsen',how='left')
zip_all_in_IRIMKT.to_excel("/home/jian/SunnyD/Otherinput/All_IRI_Mkt_with_Zips.xlsx",index=False)
